In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import os
import glob as gb
import cv2
import PIL
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from keras.models import Model

In [24]:
trainpath = '/kaggle/input/riceleafdiseasedataset/dataset/train'
testpath = '/kaggle/input/riceleafdiseasedataset/dataset/test'

# Image Processing

In [25]:
# Image Processing - Training Data
new_size = 224
train_images = []
train_labels = []
class_disease = {'BacterialBlight': 0, 'Blast': 1, 'BrownSpot': 2, 'Tungro': 3}

for i in os.listdir(trainpath):
    if i in class_disease:
        print("Entering the folder:", i)
        files = gb.glob(pathname=str(trainpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(trainpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder:", len(files))
        for j in files:
            image_raw = cv2.imread(j)
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
            resize_image = cv2.resize(image, (new_size, new_size))
            train_images.append(list(resize_image))
            train_labels.append(class_disease[i])

# Image Processing - Testing Data
new_size = 224
test_images = []
test_labels = []

for i in os.listdir(testpath):
    if i in class_disease:
        print("Entering to the folder name:", i)
        files = gb.glob(pathname=str(testpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(testpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder is", len(files))
        for j in files:
            image_raw = cv2.imread(j)
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
            resize_image = cv2.resize(image, (new_size, new_size))
            test_images.append(list(resize_image))
            test_labels.append(class_disease[i])

def list_to_array_train(train_images, train_labels):
    return np.array(train_images), np.array(train_labels)

X_train, y_train = list_to_array_train(train_images, train_labels)

def list_to_array_test(test_images, test_labels):
    return np.array(test_images), np.array(test_labels)

X_test, y_test = list_to_array_test(test_images, test_labels)

print(X_train.shape)
print("*" * 20)
print(y_train.shape)
print("*" * 20)
print(X_test.shape)
print(y_test.shape)

def keras_to_categorical(y_train, y_test):
    return to_categorical(y_train), to_categorical(y_test)

y_train1 = y_train
y_test1 = y_test
y_train, y_test = keras_to_categorical(y_train, y_test)

y_train1.shape, y_test1.shape

def convert_one_hot_to_categorical(one_hot_labels):
    return np.argmax(one_hot_labels, axis=1)

gpus = tf.config.list_physical_devices('GPU')
print(f"Num GPUs Available: {len(gpus)}")

if len(gpus) < 2:
    print("Not enough GPUs available, ensure your environment is configured correctly")
else:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    strategy = tf.distribute.MirroredStrategy()

Entering the folder: Tungro
Number of images in the folder: 1108
Entering the folder: BacterialBlight
Number of images in the folder: 1384
Entering the folder: Blast
Number of images in the folder: 1240
Entering the folder: BrownSpot
Number of images in the folder: 1400
Entering to the folder name: Tungro
Number of images in the folder is 200
Entering to the folder name: BacterialBlight
Number of images in the folder is 200
Entering to the folder name: Blast
Number of images in the folder is 200
Entering to the folder name: BrownSpot
Number of images in the folder is 200
(5132, 224, 224, 3)
********************
(5132,)
********************
(800, 224, 224, 3)
(800,)
Num GPUs Available: 2


In [26]:
# Function to convert one-hot encoded labels to categorical labels
def convert_one_hot_to_categorical(one_hot_labels):
    return np.argmax(one_hot_labels, axis=1)

In [27]:
# Deep Feature Extraction - VGG19
def model_vgg19():
    VGG_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in VGG_model.layers:
        layer.trainable = False
    feature = GlobalAveragePooling2D()(VGG_model.output)
    output = Model(inputs=VGG_model.input, outputs=feature)
    return output

# Deep Feature Extraction - ResNet50
def model_resnet50():
    Resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in Resnet_model.layers:
        layer.trainable = False
    feature = GlobalAveragePooling2D()(Resnet_model.output)
    output = Model(inputs=Resnet_model.input, outputs=feature)
    return output


In [28]:
# LBP

from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray

# Parameters for LBP
radius = 3
n_points = 8 * radius

def compute_lbp_features(image):
    if image.ndim == 3 and image.shape[2] == 3:  # Check if the image is RGB
        gray_image = rgb2gray(image)
    elif image.ndim == 2:  # If the image is already grayscale
        gray_image = image
    else:
        raise ValueError(f"Unexpected image shape: {image.shape}")
    
    lbp = local_binary_pattern(gray_image, n_points, radius, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)  # Normalize the histogram
    return hist

def extract_lbp_features(X):
    return np.array([compute_lbp_features(img) for img in X])

train_feature_lbp = extract_lbp_features(X_train)
test_feature_lbp = extract_lbp_features(X_test)


/opt/conda/lib/python3.10/site-packages/skimage/feature/texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


# VGG19 + EfficientNetB3 + LBP + Random Forest Classifier

In [29]:
from sklearn.ensemble import RandomForestClassifier

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():  
        model_FE_19 = model_vgg19()
        model_FE_19.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_19 = model_FE_19.predict(X_train)
        test_feature_19 = model_FE_19.predict(X_test)
        
        model_FE_50 = model_resnet50()
        model_FE_50.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_50 = model_FE_50.predict(X_train)
        test_feature_50 = model_FE_50.predict(X_test)
        
        final_train = np.hstack((train_feature_50, train_feature_19, train_feature_lbp))
        final_test = np.hstack((test_feature_50, test_feature_19, test_feature_lbp))

        rf = RandomForestClassifier()
        rf = rf.fit(final_train, y_train)
        test_pred = rf.predict(final_test)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Calculate average metrics
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

# Print average metrics
print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'VGG19+EfficientNetB3+LBP+RF.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 25s 114ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 20s 97ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 1 - Accuracy: 0.98375000, Recall: 0.98375000, Precision: 0.99626866, F1 Score: 0.98984597, AUC: 0.99125000
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 20s 97ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 2 - Accuracy: 0.97500000, Recall: 0.97500000, Precision: 1.00000000, F1 Score: 0.98722705, AUC: 0.98750000
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 95ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 20s 98ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 3 - Accuracy: 0.97625000, Recall: 0.97625000, Precision: 1.00000000, F1 Score: 0.98792268, AUC: 0.98812500
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 95ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 20s 9

# VGG19 + EfficientNetB3 + LBP + Decision Tree Classifier

In [30]:
from sklearn.tree import DecisionTreeClassifier

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():  
        model_FE_19 = model_vgg19()
        model_FE_19.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_19 = model_FE_19.predict(X_train)
        test_feature_19 = model_FE_19.predict(X_test)
        
        model_FE_50 = model_resnet50()
        model_FE_50.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_50 = model_FE_50.predict(X_train)
        test_feature_50 = model_FE_50.predict(X_test)
        
        final_train = np.hstack((train_feature_50, train_feature_19, train_feature_lbp))
        final_test = np.hstack((test_feature_50, test_feature_19, test_feature_lbp))

        # Define and train Decision Tree Classifier
        dt = DecisionTreeClassifier()
        dt.fit(final_train, y_train)
        test_pred = dt.predict(final_test)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'VGG19+EfficientNetB3+LBP+DTC.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 92ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step
Run 1 - Accuracy: 0.91750000, Recall: 0.91750000, Precision: 0.91739405, F1 Score: 0.91733206, AUC: 0.94500000
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 27s 146ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step
Run 2 - Accuracy: 0.92500000, Recall: 0.92500000, Precision: 0.92514834, F1 Score: 0.92497552, AUC: 0.95000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 20s 97ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 3 - Accuracy: 0.94000000, Recall: 0.94000000, Precision: 0.94000109, F1 Score: 0.93992616, AUC: 0.96000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 20s 96ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step
Ru

# VGG19 + EfficientNetB3 + LBP + KNN Classifier

In [31]:
from sklearn.neighbors import KNeighborsClassifier

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test, num):
    with strategy.scope():  
        model_FE_19 = model_vgg19()
        model_FE_19.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_19 = model_FE_19.predict(X_train)
        test_feature_19 = model_FE_19.predict(X_test)
        
        model_FE_50 = model_resnet50()
        model_FE_50.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_50 = model_FE_50.predict(X_train)
        test_feature_50 = model_FE_50.predict(X_test)
        
        final_train = np.hstack((train_feature_50, train_feature_19, train_feature_lbp))
        final_test = np.hstack((test_feature_50, test_feature_19, test_feature_lbp))

        # Define and train KNN Classifier
        knn = KNeighborsClassifier(n_neighbors=num)
        knn.fit(final_train, y_train)
        test_pred = knn.predict(final_test)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}
num = 5

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test, num)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")
    num = num+1

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'VGG19+EfficientNetB3+LBP+KNN.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 93ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step
Run 1 - Accuracy: 0.99625000, Recall: 0.99625000, Precision: 0.99628097, F1 Score: 0.99624993, AUC: 0.99750000
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 93ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step
Run 2 - Accuracy: 0.98750000, Recall: 0.98750000, Precision: 0.99875622, F1 Score: 0.99306655, AUC: 0.99354167
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 92ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 3 - Accuracy: 0.99500000, Recall: 0.99500000, Precision: 0.99506164, F1 Score: 0.99499978, AUC: 0.99666667
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 93ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step
Run

# VGG19 + EfficientNetB3 + LBP + SVM Classifier

In [32]:
from sklearn.svm import SVC

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():  
        y_train_cat = convert_one_hot_to_categorical(y_train)
        y_test_cat = convert_one_hot_to_categorical(y_test)
        
        model_FE_19 = model_vgg19()
        model_FE_19.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_19 = model_FE_19.predict(X_train)
        test_feature_19 = model_FE_19.predict(X_test)
        
        model_FE_50 = model_resnet50()
        model_FE_50.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_50 = model_FE_50.predict(X_train)
        test_feature_50 = model_FE_50.predict(X_test)
        
        final_train = np.hstack((train_feature_50, train_feature_19, train_feature_lbp))
        final_test = np.hstack((test_feature_50, test_feature_19, test_feature_lbp))

        # Define and train SVM Classifier
        svm = SVC(probability=True)
        svm.fit(final_train, y_train_cat)
        test_pred = svm.predict(final_test)
        test_pred_proba = svm.predict_proba(final_test)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test_cat, test_pred)
        recall = recall_score(y_test_cat, test_pred, average='weighted')
        precision = precision_score(y_test_cat, test_pred, average='weighted')
        f1 = f1_score(y_test_cat, test_pred, average='weighted')
        auc = roc_auc_score(y_test_cat, test_pred_proba, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'VGG19+EfficientNetB3+LBP+SVM.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 95ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 20s 95ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step
Run 1 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 95ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step
Run 2 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 94ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step
Run 3 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 93ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run

# VGG19 + EfficientNetB3 + LBP + XGBoost Classifier

In [33]:
from xgboost import XGBClassifier

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():  
        y_train_cat = convert_one_hot_to_categorical(y_train)
        y_test_cat = convert_one_hot_to_categorical(y_test)
        
        model_FE_19 = model_vgg19()
        model_FE_19.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_19 = model_FE_19.predict(X_train)
        test_feature_19 = model_FE_19.predict(X_test)
        
        model_FE_50 = model_resnet50()
        model_FE_50.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_50 = model_FE_50.predict(X_train)
        test_feature_50 = model_FE_50.predict(X_test)
        
        final_train = np.hstack((train_feature_50, train_feature_19, train_feature_lbp))
        final_test = np.hstack((test_feature_50, test_feature_19, test_feature_lbp))

        # Define and train XGB Classifier
        xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
        xgb.fit(final_train, y_train_cat)
        test_pred = xgb.predict(final_test)
        test_pred_proba = xgb.predict_proba(final_test)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test_cat, test_pred)
        recall = recall_score(y_test_cat, test_pred, average='weighted')
        precision = precision_score(y_test_cat, test_pred, average='weighted')
        f1 = f1_score(y_test_cat, test_pred, average='weighted')
        auc = roc_auc_score(y_test_cat, test_pred_proba, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'VGG19+EfficientNetB3+LBP+XGB.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 92ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Run 1 - Accuracy: 0.99625000, Recall: 0.99625000, Precision: 0.99626847, F1 Score: 0.99624995, AUC: 0.99998542
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 93ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step
Run 2 - Accuracy: 0.99625000, Recall: 0.99625000, Precision: 0.99626847, F1 Score: 0.99624995, AUC: 0.99998542
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 95ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 19s 92ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step
Run 3 - Accuracy: 0.99625000, Recall: 0.99625000, Precision: 0.99626847, F1 Score: 0.99624995, AUC: 0.99998542
161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 22s 103ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
Ru